# misura dello stoke shift prendendo uno spettro di eccitazione e uno di emissione:

Il campione preso in considerazione è Rodamina 6G + H2O


In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import os
import sys
import plotly.graph_objects as go 
dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')
from labbiofisica import Interpolazione, final_val
from scipy.optimize import curve_fit

In [2]:
# # real rodamina Jenni
# R6G_emission = pd.read_csv(r'.\data\Rhodamine 6G_emission.csv', sep=',')
# R6G_absorbance = pd.read_csv(r'.\data\Rhodamine 6G_absorbance.csv', sep=',')
# R6G_excitation = pd.read_csv(r'.\data\Rhodamine 6G_excitation.csv', sep=',')

# fondo_ecc = pd.read_csv(r'./data/fondoH20Sspettrofluorimetro.csv')

Y_PEAK_ERR = 0.0045 # analizzato in studio_errori_spettrofluorimetro.py

IMPORT DATI

In [3]:
filename = 'data/stokes_shift_05.csv'
data = pd.read_csv(filename, sep=',',header=1,nrows=151)
# Drop the 'Unnamed: 4' column as it contains no useful data
data = data.drop(columns=['Unnamed: 4'])
data.tail()

,Wavelength (nm),Intensity (a.u.),Wavelength (nm).1,Intensity (a.u.).1
146,NaN,NaN,545.969971,420.667816
147,NaN,NaN,547.010010,377.603302
148,NaN,NaN,548.049988,338.718384
149,NaN,NaN,548.950012,303.112579
150,NaN,NaN,550.000000,268.755066


In [4]:
ecc_lambda = data['Wavelength (nm).1'].dropna().to_numpy()
ecc_int = data['Intensity (a.u.).1'].dropna().to_numpy()

emi_lambda = data['Wavelength (nm)'].dropna().to_numpy()
emi_int = data['Intensity (a.u.)'].dropna().to_numpy()

norm_factor_exc = np.max(ecc_int)
norm_factor_em = np.max(emi_int)

fig = go.Figure()
fig.add_trace(go.Scatter(x=ecc_lambda, y=ecc_int/norm_factor_exc *100, mode='lines+markers', name='Excitation', line=dict(color='blue')))
# fig.add_trace(go.Scatter(x=R6G_excitation['Wavelength'], y=R6G_excitation['Relative intensity'], mode='lines', name='Excitation theoric', line=dict(color='purple')))
fig.add_trace(go.Scatter(x=emi_lambda, y=emi_int/norm_factor_em *100, mode='lines+markers', name='Emission', line=dict(color='red')))
# fig.add_trace(go.Scatter(x=R6G_emission['Wavelength'], y=R6G_emission['Relative intensity'], mode='lines', name='Emission theoric', line=dict(color='orange')))
fig.update_layout(
    # title='Stokes Shift',
    xaxis_title='Wavelength (nm)',
    yaxis_title='Intensity (a.u.)',
    legend=dict(x=0.1, y=0.9),
    font=dict(size=14),
    width=800,
    height=600,
)
fig.show()


FIT PARABOLICO DEI PICCHI

In [5]:
def parabola(x, x0,y0,a): # a positivo per picchi
    return y0 - a*(x-x0)**2

In [6]:
max_index_ecc_int = np.argmax(ecc_int)
max_index_emi_int = np.argmax(emi_int)

d = 5
y = ecc_int[max_index_ecc_int-d:max_index_ecc_int+d]
x = ecc_lambda[max_index_ecc_int-d:max_index_ecc_int+d]
sigmay = y * Y_PEAK_ERR
guess = [ecc_lambda[max_index_ecc_int], ecc_int[max_index_ecc_int], 2]
popt_ecc, pcov_ecc = curve_fit(parabola, x, y, p0=guess, sigma=sigmay, absolute_sigma=True)
error_ecc = np.sqrt(np.diag(pcov_ecc))
res = dict(zip(['λecc', 'y0', 'a'], popt_ecc))
print('ECCITAZIONE: ')
print(res)

y = emi_int[max_index_emi_int-d:max_index_emi_int+d]
x = emi_lambda[max_index_emi_int-d:max_index_emi_int+d]
sigmay = y * Y_PEAK_ERR
guess = [emi_lambda[max_index_emi_int], emi_int[max_index_emi_int], 2]
popt_emi, pcov_emi = curve_fit(parabola, x, y, p0=guess,sigma=sigmay, absolute_sigma=True)
error_emi = np.sqrt(np.diag(pcov_emi))
res = dict(zip(['λemi', 'y0', 'a'], popt_emi))
print('EMISSIONE: ')
print(res)

ECCITAZIONE: 
{'λecc': np.float64(526.7285798603273), 'y0': np.float64(962.4133734773484), 'a': np.float64(2.973017602828064)}
EMISSIONE: 
{'λemi': np.float64(553.9520035226043), 'y0': np.float64(963.3854183589305), 'a': np.float64(2.3233105574553363)}


STOKE SHIFT = $\Delta\lambda$ = PICCO EMISSIONE - PICCO ECCITAZIONE = $\lambda_{emi} - \lambda_{exc}$

NOTA: lo stoke shift è influenzato dagli effetti di solvente

In [7]:
# STOKE SHIFT
print('stoke shift:')
delta_lambda = np.abs(popt_ecc[0] - popt_emi[0])
sigma_delta_lambda = np.sqrt(error_ecc[0]**2 + error_emi[0]**2)
print('Δλ: ', final_val(delta_lambda, sigma_delta_lambda,1,udm='nm'))
stokes_shift = 1240/popt_ecc[0] - 1240/popt_emi[0]
sigma_delta_lambda = np.sqrt((1240/popt_ecc[0]**2 * error_ecc[0])**2 + (1240/popt_emi[0]**2 * error_emi[0])**2)
print('ΔE: ', final_val(stokes_shift, sigma_delta_lambda,4,udm='eV'))

stoke shift:
Δλ:  27.2 ± 0.1 nm
ΔE:  0.1157 ± 0.0006 eV


In [9]:
x_ecc = np.linspace(ecc_lambda[max_index_ecc_int-d], ecc_lambda[max_index_ecc_int+d], 1000)
y_ecc = parabola(x_ecc, *popt_ecc)

x_emi = np.linspace(emi_lambda[max_index_emi_int-d], emi_lambda[max_index_emi_int+d], 1000)
y_emi = parabola(x_emi, *popt_emi)

fig = go.Figure()
fig.add_trace(go.Scatter(x=ecc_lambda, y=ecc_int,
                    mode='lines+markers',
                    name='Excitation'))
fig.add_trace(go.Scatter(x=emi_lambda, y=emi_int,
                    mode='lines+markers',
                    name='Emission'))

fig.add_trace(go.Scatter(x=x_ecc, y=y_ecc,
                    mode='lines',
                    name='Excitation Fit',
                    line=dict(color='blue', width=5)))

fig.add_trace(go.Scatter(x=x_emi, y=y_emi,
                    mode='lines',
                    name='Emission Fit',
                    line=dict(color='red', width=5)))

fig.add_shape(
    name='Stokes Shift',
    type="rect",
    x0=ecc_lambda[max_index_ecc_int], x1=emi_lambda[max_index_emi_int],
    y0=0, y1=1000,
    fillcolor="green",
    opacity=0.3,
    layer="below",
    line_width=0,
)

fig.update_layout(
    xaxis_title='Wavelength (nm)',
    yaxis_title='Intensity (a.u.)',
    height=600,
    width=800,
    yaxis=dict(range=[0, 1000]),
    legend=dict(x=0.1, y=0.9),
    font=dict(size=14))

fig.write_html(dir_path + "/html/stokeshift.html")
fig.write_image(dir_path + "/images/stokeshift.png")
fig.show()